In [1]:
# Load MVBench dataset

import argparse
import itertools
import json
import os
import random
import time
from functools import partial

import cv2
import imageio
import torchvision.transforms as T
import numpy as np
import torch
from decord import VideoReader, cpu
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms.functional import InterpolationMode
from tqdm import tqdm



#shares/SR004.nfs2/lipengyi/1_Checked_idea

data_list = {
    'Action Sequence': ('action_sequence.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/star/star/Charades_v1_480/', 'video', True),
    # has start & end
    'Action Prediction': ('action_prediction.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/star/star/Charades_v1_480/', 'video', True),
    'Action Antonym': ('action_antonym.json', '/home/jovyanshares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/ssv2_video/ssv2_video/', 'video', False),
    'Fine-grained Action': ('fine_grained_action.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/Moments_in_Time_Raw/Moments_in_Time_Raw/videos/', 'video', False),
    'Unexpected Action': ('unexpected_action.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/FunQA_test/FunQA_test/test/', 'video', False),
    'Object Existence': ('object_existence.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/clevrer/clevrer/video_validation/', 'video', False),
    'Object Interaction': ('object_interaction.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/star/star/Charades_v1_480/', 'video', True),
    # has start & end
    'Object Shuffle': ('object_shuffle.json', '/home/jovyanshares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/perception/perception/videos/', 'video', False),
    'Moving Direction': ('moving_direction.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/clevrer/clevrer/video_validation/', 'video', False),
    'Action Localization': ('action_localization.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/sta/sta/sta_video/', 'video', True),
    # has start & end
    'Scene Transition': ('scene_transition.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/scene_qa/scene_qa/video/', 'video', False),
    'Action Count': ('action_count.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/perception/perception/videos/', 'video', False),
    'Moving Count': ('moving_count.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/clevrer/clevrer/video_validation/', 'video', False),
    'Moving Attribute': ('moving_attribute.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/clevrer/clevrer/video_validation/', 'video', False),
    'State Change': ('state_change.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/perception/perception/videos/', 'video', False),
    'Fine-grained Pose': ('fine_grained_pose.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/nturgbd/nturgbd/', 'video', False),
    'Character Order': ('character_order.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/perception/perception/videos/', 'video', False),
    'Egocentric Navigation': ('egocentric_navigation.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/vlnqa/vlnqa/', 'video', False),
    'Episodic Reasoning': ('episodic_reasoning.json', '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/tvqa/tvqa/frames_fps3_hq/', 'frame', True),
    # has start & end, read frame
    'Counterfactual Inference': (
    'counterfactual_inference.json', '1_Checked_idea/MVBench/video/clevrer/clevrer/video_validation/', 'video', False),
    
}

data_dir = '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/json'

class MVBenchDataset(torch.utils.data.Dataset):

    def __init__(self, data_dir, data_list, prompt, question_prompt, num_segments=16, input_size=224,
                 dynamic_image_size=False, use_thumbnail=False, max_num=6):
        
        self.data_list = []

        self.prompt = prompt
        self.question_prompt = question_prompt
        self.input_size = input_size
        self.num_segments = num_segments
        self.dynamic_image_size = dynamic_image_size
        self.use_thumbnail = use_thumbnail
        self.max_num = max_num
        #self.transform = build_transform(is_train=False, input_size=input_size)
        
        for k, v in data_list.items():
            with open(os.path.join(data_dir, v[0]), 'r') as f:
                json_data = json.load(f)
            for data in json_data:
                video_path = os.path.join(v[1], data['video'])
                if os.path.exists(video_path):  # 只添加存在的视频文件对应的数据
                    self.data_list.append({
                        'task_type': k,
                        'prefix': v[1],
                        'data_type': v[2],
                        'bound': v[3],
                        'data': data
                    })

        self.decord_method = {
            'video': self.read_video,
            'gif': self.read_gif,
            'frame': self.read_frame,
        }

    def __len__(self):
        return len(self.data_list)

    def __str__(self):
        len_list = {}
        option_list = {}
        for data in self.data_list:
            if data['task_type'] not in len_list:
                len_list[data['task_type']] = 0
            len_list[data['task_type']] += 1
            if data['task_type'] not in option_list:
                option_list[data['task_type']] = 0
            option_list[data['task_type']] += len(data['data']['candidates'])

        correct = 0
        total = 0
        res = f'There are {len(self.data_list)} videos as follow:\n'
        for k, v in len_list.items():
            correct += len_list[k]
            total += option_list[k]
            res += f'{v} for {k} ({option_list[k]} options => {len_list[k] / option_list[k] * 100:.2f}%)\n'
            correct = correct + 1 / option_list[k]
        res += f'Total random accuracy: {correct / total * 100:.2f}%'
        return res.rstrip()

    def get_index(self, bound, fps, max_frame, first_idx=0):
        if bound:
            start, end = bound[0], bound[1]
        else:
            start, end = -100000, 100000
        start_idx = max(first_idx, round(start * fps))
        end_idx = min(round(end * fps), max_frame)
        seg_size = float(end_idx - start_idx) / self.num_segments
        frame_indices = np.array([
            int(start_idx + (seg_size / 2) + np.round(seg_size * idx))
            for idx in range(self.num_segments)
        ])
        return frame_indices

    def read_video(self, video_path, bound=None):
        vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
        max_frame = len(vr) - 1
        fps = float(vr.get_avg_fps())

        images_group = list()
        frame_indices = self.get_index(bound, fps, max_frame, first_idx=0)
        for frame_index in frame_indices:
            img = Image.fromarray(vr[frame_index].asnumpy())
            images_group.append(img)

        return images_group

    def read_gif(self, video_path, bound=None, fps=25):
        gif = imageio.get_reader(video_path)
        max_frame = len(gif) - 1

        images_group = list()
        frame_indices = self.get_index(bound, fps, max_frame, first_idx=0)
        for index, frame in enumerate(gif):
            if index in frame_indices:
                img = cv2.cvtColor(frame, cv2.COLOR_RGBA2RGB)
                img = Image.fromarray(img)
                images_group.append(img)

        return images_group

    def read_frame(self, video_path, bound=None, fps=3):
        max_frame = len(os.listdir(video_path))
        images_group = list()
        frame_indices = self.get_index(bound, fps, max_frame, first_idx=1)  # frame_idx starts from 1
        for frame_index in frame_indices:
            img = Image.open(os.path.join(video_path, f'{frame_index:05d}.jpg'))
            images_group.append(img)

        return images_group

    def qa_template(self, data):
        question = f"Question: {data['question']}\n"
        question += 'Options:\n'
        answer = data['answer']
        answer_idx = -1
        for idx, c in enumerate(data['candidates']):
            question += f"({chr(ord('A') + idx)}) {c}\n"
            if c == answer:
                answer_idx = idx
        question = question.rstrip()
        answer = f"({chr(ord('A') + answer_idx)}) {answer}"
        return question, answer

    def __getitem__(self, idx):
        data_item = self.data_list[idx]
        video_path = os.path.join(data_item['prefix'], data_item['data']['video'])
        
        if not os.path.exists(video_path):
            print(f"Warning: Video file not found at {video_path}, skipping this item.")
            return None  # 或者返回一个特定的占位符数据结构
    
        decord_method = self.decord_method[data_item['data_type']]
        bound = None
        if data_item['bound']:
            bound = (data_item['data']['start'], data_item['data']['end'])
    
        image_list = decord_method(video_path, bound)
        if not image_list:  # 检查返回的列表是否为空
            print(f"Warning: No images were extracted from video at {video_path}, skipping this item.")
            return None  # 或者返回一个特定的占位符数据结构
    
        # 处理图像和问题-答案数据
        #special_tokens = '\n'.join(['Frame{}: <image>'.format(i + 1) for i in range(len(image_list))])
        question, answer = self.qa_template(data_item['data'])
        question = self.prompt + '\n' + question + self.question_prompt
        #question = question + self.question_prompt
    
        #raw_images = []
        #num_patches_list = []
        #pixel_values = []
        #for image in image_list:
            #raw_images.append(image)
            #if self.dynamic_image_size:
                #patches = dynamic_preprocess(image, image_size=self.input_size,
                                             #use_thumbnail=self.use_thumbnail,
                                             #max_num=self.max_num)
            #else:
                #patches = [image]
            #num_patches_list.append(len(patches))
            #pixel_values.extend([self.transform(patch) for patch in patches])
    
        #if not pixel_values:  # 检查是否有有效的像素值数据
            #return None  # 返回占位符或None
    
        #pixel_values = torch.stack(pixel_values)
    
        return {
            'question': question,
            'video_path': video_path,
            #'pixel_values': pixel_values,
            'answer': answer,
            'task_type': data_item['task_type']
        }

In [2]:
prompt = 'Carefully watch the video and pay attention to the cause and sequence of events, the detail and movement of objects, and the action and pose of persons. Based on your observations, select the best option that accurately addresses the question.\n'
question_prompt = '\nOnly give the best option.'

dataset = MVBenchDataset(
    data_dir=data_dir,          
    data_list=data_list, 
    prompt = prompt,
    question_prompt=question_prompt,
    num_segments=16,              
    input_size=448,              
    dynamic_image_size=True,     
    use_thumbnail=False,          
    max_num=6                     
)

In [3]:
print(dataset)

There are 3186 videos as follow:
188 for Action Sequence (752 options => 25.00%)
200 for Action Prediction (800 options => 25.00%)
200 for Fine-grained Action (800 options => 25.00%)
200 for Unexpected Action (800 options => 25.00%)
198 for Object Existence (594 options => 33.33%)
200 for Object Interaction (800 options => 25.00%)
200 for Moving Direction (800 options => 25.00%)
200 for Action Localization (800 options => 25.00%)
200 for Scene Transition (800 options => 25.00%)
200 for Action Count (600 options => 33.33%)
200 for Moving Count (800 options => 25.00%)
200 for Moving Attribute (601 options => 33.28%)
200 for State Change (600 options => 33.33%)
200 for Character Order (600 options => 33.33%)
200 for Egocentric Navigation (800 options => 25.00%)
200 for Episodic Reasoning (1000 options => 20.00%)
Total random accuracy: 26.67%


In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'


In [5]:
model_path = "/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft"#"/home/jinyang06/models/VideoLaVIT-v1/language_model_sft"
model_dtype='bf16'

max_video_clips = 16
device_id = 0
torch.cuda.set_device(device_id)
device = torch.device('cuda')

seed = 42
#torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# For Multi-Modal Understanding
#runner = build_model(model_path=model_path, model_dtype=model_dtype, understanding=True, 
#        device_id=device_id, use_xformers=False, max_video_clips=max_video_clips,)

In [7]:
dict_ = torch.load("/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/custom_ckpts/visual_tokenizer/model.pth")
#runner.model.model.visual_tokenizer.load_state_dict(dict_)

In [8]:
import transformers

tokenizer = transformers.LlamaTokenizer.from_pretrained(model_path, use_fast=False, padding_side='left')
tokenizer.pad_token = tokenizer.unk_token

In [9]:
import torch
from torch.utils.data import DataLoader
from functools import partial


def collate_fn(batches, tokenizer):
    
    questions = [_['question'] for _ in batches]
    video_path = [_['video_path'] for _ in batches]
    answer = [_['answer'] for _ in batches]
    task_types = [_['task_type'] for _ in batches]
    
    return questions, video_path, answer, task_types

dataloader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=1,
        num_workers=1,
        pin_memory=True,
        drop_last=False,
        collate_fn=partial(collate_fn, tokenizer=tokenizer)
    )

first_batch = next(iter(dataloader))


question = first_batch[0]
video_path = first_batch[1]
answer = first_batch[2]
task_type = first_batch[3]

print(question)
print(answer)
print(video_path)
print(task_type)

['Carefully watch the video and pay attention to the cause and sequence of events, the detail and movement of objects, and the action and pose of persons. Based on your observations, select the best option that accurately addresses the question.\n\nQuestion: What happened after the person took the food?\nOptions:\n(A) Ate the medicine.\n(B) Tidied up the blanket.\n(C) Put down the cup/glass/bottle.\n(D) Took the box.\nOnly give the best option.']
['(A) Ate the medicine.']
['/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/star/star/Charades_v1_480/ZS9XR.mp4']
['Action Sequence']


In [10]:
from transformers import AutoModel, AutoTokenizer, AutoProcessor

video_path = '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/star/star/Charades_v1_480/ZS9XR.mp4'
prompt = "What happened after the person took the food?\nOptions:\n(A) Ate the medicine.\n(B) Tidied up the blanket.\n(C) Put down the cup/glass/bottle.\n(D) Took the box.\nOnly give the best option."
answer = "(A) Ate the medicine."
output = runner({"video": video_path, "text_input": question[0].split('Question:')[1]}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be

::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::


In [10]:
output.strip()

'C. Put down the cup/glass/bottle.'

In [ ]:
generation_config = dict(max_new_tokens=1024, do_sample=False)
epoch = 1

y_pred = []
y_real = []

progress_bar = tqdm(
        dataloader, total=len(dataloader), desc=f"Epoch {epoch}/1"
    )

for step, batch in enumerate(progress_bar, start=1):
    print (step)
    question = batch[0]
    video_path = batch[1]
    answer = batch[2]
    task_type = batch[3]

    # print(pixel_values.size())
    # print(question)
    # print(answer)
    # print(num_patches_list)
    # print(task_type)
    
    # with autocast():
        # 执行推理
    outputs = runner({"video": video_path, "text_input": question[0].split('Question: ')[1]}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]

    y_pred.append(outputs.strip())
    y_real.append(answer[0].strip())
    if ((step-1)%100 == 0):
        print("="*20 + "output" + "="*20)
        print(outputs)
        print("="*20 + "real answers" + "="*20)
        print(answer[0], flush = True)

Epoch 1/1:   0%|          | 0/3186 [00:00<?, ?it/s]

1
====================output====================
(C) Put down the cup/glass/bottle.
====================real answers====================
(A) Ate the medicine.


Epoch 1/1:   0%|          | 1/3186 [00:07<7:04:13,  7.99s/it]

2


Epoch 1/1:   0%|          | 2/3186 [00:11<4:51:35,  5.49s/it]

...

2719


Epoch 1/1:  85%|████████▌ | 2719/3186 [2:37:39<1:25:15, 10.95s/it]

2720


Epoch 1/1:  85%|████████▌ | 2720/3186 [2:37:45<1:13:45,  9.50s/it]

2721


In [9]:
y_preds = [i.split('.')[0] for i in y_pred]

result = [1 if i == j else 0 for i, j in zip(y_real, y_preds)]

print("VideoLAVIT-8.5B-MVBench, accuracy is: " + str(sum(result) / len(result)))

InternVL-2B-8-frames-MVBench, accuracy is: 0.5443740095087163


In [ ]:
/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/star/star/Charades_v1_480/ZS9XR.mp4 What happened after the person took the food?

In [21]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'


In [10]:
model_path = "/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft"#"/home/jinyang06/models/VideoLaVIT-v1/language_model_sft"
model_dtype='bf16'

max_video_clips = 16
device_id = 0
torch.cuda.set_device(device_id)
device = torch.device('cuda')

seed = 42
#torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# For Multi-Modal Understanding
runner = build_model(model_path=model_path, model_dtype=model_dtype, understanding=True, 
        device_id=device_id, use_xformers=False, max_video_clips=max_video_clips,)

Loading Video LaVIT Model Weight from /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft, model precision: bf16
Not used {}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft were not used when initializing VideoLaVITLlamaForCausalLM: ['model.motion_tokenizer.quantize.cluster_size', 'model.motion_tokenizer.quantize.embedding.initted', 'model.motion_tokenizer.quantize.embedding.embed_avg', 'model.motion_tokenizer.quantize.embedding.cluster_size']
- This IS expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The Visual Vocab Size is 16384
The llama tokenizer vocab size is 32000
The maximal clip number is 16


In [12]:
video_path = '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/star/star/Charades_v1_480/EY6P4.mp4'
prompt = "What happened before the person watched at the book?"
answer = "He ate the medicine."
output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

prompt A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: <video>
What happened before the person watched at the book? ASSISTANT:
The video shows a person walking into the frame while talking with someone. Later, the person pointed at a book and walked towards the camera. But the video does not provide more information regarding the previous actions.


In [13]:
video_path = '/home/jovyan/shares/SR004.nfs2/lipengyi/1_Checked_idea/MVBench/video/star/star/Charades_v1_480/EY6P4.mp4'
prompt = "What happened on the video?"
answer = "What happens on the video?"
output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

prompt A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: <video>
What happened on the video? ASSISTANT:
The video depicts a man in a black shirt who stands inside a room holding papers and interacts with a laptop. He opens the door and walks out.
